In [3]:
import tensorflow as tf
import datetime
import os
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.models import Model
import pandas as pd 
import numpy as np



# student shape 22025
student = pd.read_csv('/content/drive/MyDrive/명륜이의 서재/FINAL_DATA/student_15.csv')
student.head()

,student_id,major
0,412737,사과대학
1,572900,소프트웨어대학
2,991018,소프트웨어대학
3,1424110,사과대학
4,1520139,사과대학


In [4]:
#delete college in each major 

def delete_text(x):
  if x == '성균융합원':
    return x
  else:
    return x[:-2]


student['major'] = student['major'].apply(lambda x: delete_text(x))
student['major'].value_counts()


공과       3770
문과       3572
사과       3223
경영       2472
경제       2205
정보통신     1450
예술       1134
자연과학     1098
사범        879
생명공학      601
소프트웨어     575
약학        370
유학        258
스포츠과학     224
성균융합원     112
의과         81
Name: major, dtype: int64

In [5]:
from sklearn.preprocessing  import LabelEncoder
import pickle
le = LabelEncoder()
le.fit(student['major'])
list(le.classes_)[:10]
len(le.classes_) #16


# output = open('Major_encoder.pkl', 'wb')
# pickle.dump(le, output)
# output.close()
le.inverse_transform([4])

#major -> number encoded 

array(['사과'], dtype=object)

In [6]:
student['major_num']=le.fit_transform(student[ 'major' ])
student.head()

# student.to_csv('student_major.csv',index=False)

,student_id,major,major_num
0,412737,사과,4
1,572900,소프트웨어,8
2,991018,소프트웨어,8
3,1424110,사과,4
4,1520139,사과,4


In [7]:

!pip install sentence_transformers
from sentence_transformers import SentenceTransformer

bert_model = SentenceTransformer('xlm-r-bert-base-nli-stsb-mean-tokens')


     |████████████████████████████████| 71kB 7.3MB/s 
     |████████████████████████████████| 1.8MB 9.6MB/s 
     |████████████████████████████████| 1.2MB 40.0MB/s 
     |████████████████████████████████| 2.9MB 66.8MB/s 
     |████████████████████████████████| 890kB 55.1MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-cp36-none-any.whl size=103068 sha256=99db2f0505b380326495bdb3e6fcb2ae533c5ae27c4ea25a3c054593369433ba
  Stored in directory: /root/.cache/pip/wheels/3d/33/d1/5703dd56199c09d4a1b41e0c07fb4e7765a84d787cbdc48ac3
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=29f1fc4484db3ae95377eeed66acd5d8673b7931319e1bca8c3d1f319de4dc6c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


100%|██████████| 1.01G/1.01G [00:12<00:00, 78.6MB/s]


In [ ]:
sentence_embeddings = bert_model.encode(student['major'])
sentence_embeddings.shape #(22024, 768)
# sentence_embeddings[0]
# type(sentence_embeddings)



(22024, 768)

In [ ]:
np.save('major_embeddings_full_name(with college).npy', sentence_embeddings) 

In [28]:
##GENRE EMBEDDING 

book = pd.read_csv('/content/drive/MyDrive/명륜이의 서재/FINAL_DATA/book_15.csv')
book.shape[0]

df = book[['book_id','genre']]
df.shape
# df['genre'].value_counts()

(133878, 2)

In [14]:
#map words with genre 

#1	Class 000 – Computer science, information & general works
# 2	Class 100 – Philosophy & psychology
# 3	Class 200 – Religion
# 4	Class 300 – Social sciences
# 5	Class 400 – Language
# 6	Class 500 – Science
# 7	Class 600 – Technology
# 8	Class 700 – Arts & recreation
# 9	Class 800 – Literature
# 10	Class 900 – History & geography

dicts = {0: 'Computer science, information & general works',
        1:'Philosophy & psychology',
        2: 'Religion', 
        3: 'Social sciences',
        4: 'Language',
        5: 'Science',
        6:'Technology',
        7:'Arts & recreation',
        8:'Literature',
        9:'History & geography'}
df['genre_word'] = df['genre'].map(dicts)
df.head()

#genre -> gerre_word
#genre -> number encoded 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,book_id,genre,genre_word
0,CM0000198,1,Philosophy & psychology
1,CM0000488,3,Social sciences
2,CM0000596,8,Literature
3,CM0000605,3,Social sciences
4,CM0000614,8,Literature


In [27]:
genre_embeddings = bert_model.encode(df['genre_word'])
genre_embeddings.shape  #(133878, 768)


(133878, 768)

In [29]:
np.save('genre_embeddings(word).npy', genre_embeddings) 

In [ ]:
#for visualization 
# Convert NumPy array of embedding into data frame
embedding_df = pd.DataFrame(sentence_embeddings)

# Save dataframe as as TSV file without any index and header
embedding_df.to_csv('output2.tsv', sep='\t', index=None, header=None)
student[['major']].to_csv('metadata2.tsv', index=False, sep='\t')